<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/performance_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_spd_matrix
import random
# import warnings
# warnings.filterwarnings('error')
import time
import cvxpy as cp
from scipy.linalg import sqrtm
import gurobipy as gp
from gurobipy import GRB

In [39]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [40]:
# !pip install import-ipynb
import import_ipynb

In [41]:
# %cd "/content/gdrive/My Drive/Colab Notebooks"
import mpc_class
import qpth_class

In [42]:
#initialize the mpc solver class 
mpc_solver=mpc_class.mpc(max_iter=30)

In [82]:
# %cd "/content"
df=pd.read_csv('test_cases.csv')
df=df.iloc[:1000]
seeds=df["seed"].astype(int)


# Test on individual problems - gurobi & cvxpy

In [83]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

In [84]:
def qp_gurobi(n,m,p,P,q,G,h,A,b):
    model = gp.Model("qp")
    model.setParam( 'OutputFlag', False )
    x={}
    for i in range(n):
        x[i]=model.addVar(lb=-GRB.INFINITY,ub=GRB.INFINITY)
    obj_0=0
    obj_1=0
    for i in range(n):
        for j in range(n):
            obj_0+=P[i][j]*x[i]*x[j]
        obj_1+=q[i]*x[i]
    model.setObjective(0.5*obj_0+obj_1)
    for i in range(m):
        cons=0
        for j in range(n):
            cons+=G[i][j]*x[j]
        model.addConstr(cons<=h[i])
    for i in range(p):
        cons=0
        for j in range(n):
            cons+=A[i][j]*x[j]
        model.addConstr(cons==b[i])
    model.optimize()
    try:
        gurobi_sol=[v.x for v in model.getVars()]
        gurobi_opt=model.objVal
    except:
        gurobi_sol=None
        gurobi_opt=None
    return np.array(gurobi_opt),gurobi_sol

In [93]:
n=6
m=6
p=3
nbatch=1
import random as r
gurobi_time=0.0
gurobi_errors=[]
ge=0
cvxpy_time=0.0
cvxpy_errors=[]
cvxpy_opt=[]
ce=0
gurobi_opt=[]
# columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([])#,columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.random()*10 for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.random()*10*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.random()*10 for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.random()*10 for i in range(p)]).reshape(p).astype(float)
#     q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
#     G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
#     h=np.array([0 for i in range(m)]).reshape(n).astype(float)
#     A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
#     b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    #get gurobi solution
    start_gur=time.time()
    y,x=qp_gurobi(n,m,p,P,q,G,h,A,b)
#     print(np.round(x,4))
#     print(np.round(y,4))
    
    
#     model = gp.Model("matrix1")
#     model.setParam( 'OutputFlag', False )
#     x = model.addMVar(shape=n, name="x",lb=-GRB.INFINITY,ub=GRB.INFINITY)
#     model.addConstr(G @ x <= h, name="ineq")
#     model.addConstr(A @ x == b, name="eq")
#     model.setMObjective(0.5*P,q.T,0.0,x,x,x, GRB.MINIMIZE)
#     # Optimize model
#     model.optimize()
    gurobi_time+=time.time()-start_gur
    re=pd.Series({"seed":s})
    if x is not None:
#         for v in model.getVars():
        for i in range(n):
            re=re.append(pd.Series({str(i)+"gur":np.round(x[i],4)}))
            # print('%s %g' % (v.varName, v.x))
        gurobi_opt.append(y)
    else:
        ge+=1
        gurobi_errors.append(s)
        for i in range(n):
            re=re.append(pd.Series({str(i)+"gur":np.nan}))
        gurobi_opt.append(0)
    
#     #get cvxpy solution
    P = sqrtm(P)
    start=time.time()
    try:
        x,y=qp_cvxpy(n,P,q,G,h,A,b)
        cvxpy_time+=time.time()-start
        cvxpy_opt.append(x)
        for i in range(n):
            if y is not None:
                re=re.append(pd.Series({str(i)+"cp":np.round(y[i],4)}))

            else:
                cvxpy_errors.append(s)
                re=re.append(pd.Series({str(i)+"cp":0}))
    except:
        cvxpy_opt.append(0)
        cvxpy_errors.append(s)
        re=re.append(pd.Series({str(i)+"cp":0}))
    
    
    results=results.append(re,ignore_index=True )
# cvxpy_errors=np.unique(cvxpy_errors)
# ce=len(cvxpy_errors)

In [94]:
print("gurobi time: ", gurobi_time)
# print("cvxpy time: ", cvxpy_time)

gurobi time:  6.41616678237915


# Test on Batch of problems - qpth & mpc

In [95]:
import random 

In [96]:
ip="csv"
df=df.iloc[:,:8]
nbatch=len(df)
nx=6
nineq=6
neq=3
Q=[]
p=[]
G=[]
h=[]
A=[]
b=[]
for i in range(nbatch):
  #generate random problems using seeds in the csv file
  seed=int(df["seed"][i])
  random.seed(seed)
  Q.append(make_spd_matrix(nx,random_state=seed).reshape(1,-1))
#   p.append([random.randint(0,9) for i in range(nx)])#.astype(float))
#   G.append([random.randint(0,9)*((-1)**random.randint(0,1)) for i in range(nineq*nx)])#.astype(float))
#   h.append([0 for i in range(nineq)])#.astype(float))
#   A.append([random.randint(0,9) for i in range(neq*nx)])#.astype(float))
#   b.append([random.randint(0,9) for i in range(neq)])#.astype(float)  )

  p.append([random.random()*10 for i in range(nx)])#.astype(float))
  G.append([random.random()*10*((-1)**random.randint(0,1)) for i in range(nineq*nx)])#.astype(float))
  h.append([0 for i in range(nineq)])#.astype(float))
  A.append([random.random()*10 for i in range(neq*nx)])#.astype(float))
  b.append([random.random()*10 for i in range(neq)])#.astype(float)  )

Q=torch.tensor(Q).view(nbatch,nx,nx).type(torch.DoubleTensor)
p=torch.tensor(p).view(nbatch,nx).type(torch.DoubleTensor)
G=torch.tensor(G).view(nbatch,nineq,nx).type(torch.DoubleTensor)
h=torch.tensor(h).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor(A).view(nbatch,neq,nx).type(torch.DoubleTensor)
b=torch.tensor(b).view(nbatch,neq).type(torch.DoubleTensor)

Q_batched=Q
p_batched=p

In [97]:
#solve using mpc solver
start=time.time()
x_mpc,_=mpc_solver.solve(Q,p,G,h,A,b)
mpc_time=time.time()-start

no of mu not converged:  360
mpc warning: Residuals not converged, need more itrations


In [98]:

#solve using qpt solver
start=time.time()
x_qpt,_,_,_=qpth_class.qpth_opt(Q,p,G,h,A,b)
qpth_time=time.time()-start
#print run times
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)

qpth warning: Returning an inaccurate and potentially incorrect solution.
mpc time:  0.5602896213531494
qpth time:  4.894780874252319


## Results Table 

In [99]:
cols_mpc=[str(i)+"mpc" for i in range(nx)]
cols_qpt=[str(i)+"qpt" for i in range(nx)]
cols_cp=[str(i)+"cp" for i in range(nx)]
cols_gur=[str(i)+"gur" for i in range(nx)]
error_qpt_cp=np.zeros(nbatch)
error_mpc_cp=np.zeros(nbatch)
error_qpt_gur=np.zeros(nbatch)
error_mpc_gur=np.zeros(nbatch)
error_cp_gur=np.zeros(nbatch)
#results of mpc and qpt as data frames
new_mpc=pd.DataFrame(x_mpc.numpy().round(4).reshape(nbatch,-1),columns=cols_mpc)
new_qpt=pd.DataFrame(x_qpt.numpy().round(4).reshape(nbatch,-1),columns=cols_qpt)
#results of cvxpy & gurobi
tmp=results.copy()
for i in range(nx):
    col_mpc=cols_mpc[i]
    col_qpt=cols_qpt[i]
    #copy mpc and qpt results to the tmp df
    tmp[col_mpc]=new_mpc[col_mpc].copy()
    tmp[col_qpt]=new_qpt[col_qpt].copy()
    #calculate difference of optimal points from cvxpy (elementwise)
    error_mpc_cp+=abs(np.round(tmp[str(i)+"cp"],4)-np.round(tmp[col_mpc],4))
    error_qpt_cp+=abs(np.round(tmp[str(i)+"cp"],4)-np.round(tmp[col_qpt],4))
    #calculate difference of optimal points from gurobi (elementwise)
    error_mpc_gur+=abs(np.round(tmp[str(i)+"gur"],4)-np.round(tmp[col_mpc],4))
    error_qpt_gur+=abs(np.round(tmp[str(i)+"gur"],4)-np.round(tmp[col_qpt],4))
    error_cp_gur+=abs(np.round(tmp[str(i)+"gur"],4)-np.round(tmp[str(i)+"cp"],4))

#calculate optimum values and store them
# cp_=torch.tensor(np.array(tmp[cols_cp])).view(nbatch,nx,1)
# cp_opt=np.round(((0.5*torch.bmm(torch.transpose(cp_,dim0=2,dim1=1),
#                                 torch.bmm(Q,cp_)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),cp_)).view(-1).numpy(),6)
tmp["cp-opt"]=np.round(cvxpy_opt,4)#cp_opt
tmp["gur-opt"]=np.round(gurobi_opt,4)

mpc=torch.tensor(np.array(tmp[cols_mpc])).view(nbatch,nx,1)
mpc_opt=np.round(((0.5*torch.bmm(torch.transpose(mpc,dim0=2,dim1=1),
                                 torch.bmm(Q,mpc)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),mpc)).view(-1).numpy(),4)
tmp["mpc-opt"]=mpc_opt

qpt=torch.tensor(np.array(tmp[cols_qpt])).view(nbatch,nx,1)
qpt_opt=np.round(((0.5*torch.bmm(torch.transpose(qpt,dim0=2,dim1=1),
                                 torch.bmm(Q,qpt)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),qpt)).view(-1).numpy(),4)
tmp["qpt-opt"]=qpt_opt

#store error values calculated in the for loop
tmp["cp-mpc-error"]=np.round(error_mpc_cp,6)
tmp["cp-mpc-er_flag"]=np.round(error_mpc_cp,6)!=0
tmp["cp-qpt-error"]=np.round(error_qpt_cp,6)
tmp["cp-qpt-er_flag"]=np.round(error_qpt_cp,6)!=0
tmp["gur-mpc-error"]=np.round(error_mpc_gur,6)
tmp["gur-mpc-er_flag"]=np.round(error_mpc_gur,6)!=0
tmp["gur-qpt-error"]=np.round(error_qpt_gur,6)
tmp["gur-qpt-er_flag"]=np.round(error_qpt_gur,6)!=0
tmp["gur-cp-error"]=np.round(error_cp_gur,6)
tmp["gur-cp-er_flag"]=np.round(error_cp_gur,6)!=0
cols_order=["seed","cp-opt","mpc-opt","qpt-opt",'gur-opt',"cp-mpc-error",
            "cp-mpc-er_flag","cp-qpt-error","cp-qpt-er_flag","gur-mpc-error","gur-mpc-er_flag",
            "gur-qpt-error","gur-qpt-er_flag","gur-cp-error","gur-cp-er_flag"]+ cols_cp+cols_mpc+cols_qpt+cols_gur
tmp=tmp[cols_order]

print("COMPARISON AGAINST GUROBI:")
print("No.of problems solved: ",len(tmp)-ge)
results_gur=tmp[~tmp["seed"].isin(gurobi_errors)].copy()
print(len(results_gur))
print("\nNo.of errors from cvxpy: ",len(results_gur[results_gur['gur-cp-error']!=0])," (",round(len(results_gur[results_gur['gur-cp-error']!=0])*100/len(results_gur),2),"%)")
print("No.of errors from MPC ",len(results_gur[results_gur['gur-mpc-error']!=0])," (",round(len(results_gur[results_gur['gur-mpc-error']!=0])*100/len(results_gur),2),"%)")
print("No.of errors from qpth ",len(results_gur[results_gur['gur-qpt-error']!=0])," (",round(len(results_gur[results_gur['gur-qpt-error']!=0])*100/len(results_gur),2),"%)")
print("______________________________________________")
print("\n\nCOMPARISON AGAINST CVXPY:")
print("No.of problems solved: ",len(tmp)-ce)
cvxpy_errors=np.unique(cvxpy_errors)
results_cp=tmp[~tmp["seed"].isin(cvxpy_errors)].copy()
print(len(results_cp))
print("No.of errors from MPC: ",len(results_cp[results_cp['cp-mpc-error']!=0])," (",round(len(results_cp[results_cp['cp-mpc-error']!=0])*100/len(results_cp),2),"%)")
print("No.of errors from qpth: ",len(results_cp[results_cp['cp-qpt-error']!=0])," (",round(len(results_cp[results_cp['cp-qpt-error']!=0])*100/len(results_cp),2),"%)")
print("______________________________________________")
print("\n\nTIME COMPARISON:")
print("gurobi time: ", gurobi_time)
print("cvxpy time: ", cvxpy_time)
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)


COMPARISON AGAINST GUROBI:
No.of problems solved:  642
642

No.of errors from cvxpy:  3  ( 0.47 %)
No.of errors from MPC  22  ( 3.43 %)
No.of errors from qpth  20  ( 3.12 %)
______________________________________________


COMPARISON AGAINST CVXPY:
No.of problems solved:  1000
641
No.of errors from MPC:  19  ( 2.96 %)
No.of errors from qpth:  17  ( 2.65 %)
______________________________________________


TIME COMPARISON:
gurobi time:  6.41616678237915
cvxpy time:  14.187674522399902
mpc time:  0.5602896213531494
qpth time:  4.894780874252319


In [100]:
u=results_gur.loc[results_gur["gur-mpc-er_flag"],np.append(cols_mpc,np.append(cols_gur,["mpc-opt","gur-opt"])) ]
u

,0mpc,1mpc,2mpc,3mpc,4mpc,5mpc,0gur,1gur,2gur,3gur,4gur,5gur,mpc-opt,gur-opt
8,1.4662,-8.3199,-3.6869,4.9170,3.8718,-1.1078,1.4662,-8.3199,-3.6869,4.9170,3.8717,-1.1078,-29.1117,-29.1125
14,10.4531,-20.7534,18.3509,-3.7013,13.9057,-13.7577,10.4531,-20.7534,18.3510,-3.7013,13.9057,-13.7577,777.5351,777.5372
89,1.2617,-0.3377,-0.0487,0.3651,0.2907,0.6700,1.1425,-0.1739,-0.0779,0.2549,0.3138,0.7019,8.5608,8.4286
116,76.0083,-118.5240,144.1918,-33.2707,28.0332,-112.1791,76.0096,-118.5259,144.1941,-33.2712,28.0336,-112.1809,138711.6857,138716.1654
142,11.3557,-18.5234,109.0042,3.0035,-107.7981,-1.3099,11.3558,-18.5236,109.0049,3.0035,-107.7989,-1.3099,6183.8701,6183.9577
162,-8.7610,7.1807,9.5307,9.4019,4.2929,-8.5675,-8.7610,7.1807,9.5308,9.4019,4.2929,-8.5676,518.8764,518.8787
203,57.7708,-89.5180,47.5900,11.4830,77.8867,-64.2987,57.7709,-89.5181,47.5901,11.4830,77.8868,-64.2988,7703.0869,7703.1084
212,13.8624,-15.7927,-39.6885,-46.5327,37.1927,52.6058,13.8623,-15.7926,-39.6882,-46.5324,37.1925,52.6055,4581.0270,4580.9706
226,12.1733,-12.7420,-1.8534,5.3629,-5.3066,4.1962,12.1732,-12.7420,-1.8534,5.3629,-5.3066,4.1962,252.0762,252.0741
244,25.9150,1.5828,8.3054,-1.0773,-25.2579,5.8040,25.9151,1.5828,8.3054,-1.0773,-25.2580,5.8040,2255.6855,2255.6995


In [ ]:
u[u["mpc-opt"]<u["gur-opt"]]

In [ ]:
u[u["mpc-opt"]>u["gur-opt"]]

In [31]:
Q.append(make_spd_matrix(nx,random_state=seed).reshape(1,-1))

AttributeError: 'Tensor' object has no attribute 'append'

In [32]:
make_spd_matrix(nx,random_state=seed).reshape(1,-1)

array([[ 0.57590251,  0.40188819,  0.18131191, -0.01467761,  0.56187803,
        -0.39983129,  0.40188819,  1.95489793,  0.285852  ,  0.01657491,
         1.9807648 , -1.01279744,  0.18131191,  0.285852  ,  0.64411307,
         0.00754676,  0.85436686, -0.44048994, -0.01467761,  0.01657491,
         0.00754676,  0.47343564,  0.24924825,  0.03113005,  0.56187803,
         1.9807648 ,  0.85436686,  0.24924825,  4.03732017, -1.96538966,
        -0.39983129, -1.01279744, -0.44048994,  0.03113005, -1.96538966,
         1.52648933]])

In [37]:
[random.random()*10 for i in range(nx)]

[6.189462708300947,
 1.9887660413572206,
 0.4062201814057498,
 8.64396409049604,
 0.2875741497702511,
 0.34007172515212236]